In [3]:
#in case we need this
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.optim as optim
from 4_Model import FTTransformer  # WE are importing class here
from torch.utils.data import Dataset, DataLoader
from data_utils import ACSDataset
from torch.utils.data import DataLoader

from sklearn.preprocessing import LabelEncoder, StandardScaler


SyntaxError: invalid decimal literal (181370680.py, line 8)

In [ ]:

# set device for gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == "cuda":
    print(f"found 1 gpu, using {device}")
else:
    print(f"found no gpu, using {device}")

In [ ]:
train_df = pd.read_csv("preprocessing_data/train_data_final_feat_no_preprocessing.csv")

train_df = train_df.drop(columns=['POVIP'])

In [ ]:
#From Marainnes preprocessing 
# Load preprocessed data from 6_Feature_Engineering_and_XGBoost
df_train = pd.read_csv('../3_Data_Preprocessing/preprocessing_data/train_engineered.csv')

df_test = pd.read_csv('../3_Data_Preprocessing/preprocessing_data/test_engineered.csv')

# Load feature lists and label encoder metadata
with open('../3_Data_Preprocessing/preprocessing_data/feature_engineering_metadata.json') as f:
    metadata = json.load(f)

all_features = metadata['all_features']
categorical_features = metadata['categorical_features']
optimal_features = metadata['optimal_features']

# Reconstruct label encoders from saved classes
label_encoders = {}
for col, classes in metadata['label_encoder_classes'].items():
    le = LabelEncoder()
    le.classes_ = np.array(classes)
    label_encoders[col] = le

# Prepare X, y
X_train = df_train[all_features].copy()
y_train = df_train['binary_target'].copy()
X_test = df_test[all_features].copy()
y_test = df_test['binary_target'].copy()

# Optimal feature subsets from dimensionality reduction
X_train_final = X_train[optimal_features].copy()
X_test_final = X_test[optimal_features].copy()

print(f'X_train: {X_train.shape}, y_train: {y_train.shape}')
print(f'X_train_final (optimal): {X_train_final.shape}')
print(f'X_test:  {X_test.shape}, y_test:  {y_test.shape}')
print(f'y_train distribution: {Counter(y_train)}')
print(f'y_test distribution:  {Counter(y_test)}')

In [ ]:
# continuous some of these are binary 
cont_cols =  ['AGEP', 'WKHP', 'disability_score', 'education_tier', 'race_ethnic_aggregate','has_insurance', 'race_white', 'race_black', 'race_asian',
                   'race_indigenous', 'race_other', 'is_latinx']

#  categorical
categorical_features = ['CIT', 'ENG', 'LANX', 'MAR', 'MIG', 'MSP', 'SEX',
                        'NATIVITY', 'ESR', 'OCCP', 'WKL', 'WRK', 'PUMA', 'CA_Region']
#target
target_col = 'poverty_risk_score'

In [ ]:
for col in cat_cols:
    train_df[col] = train_df[col].fillna(-1)
for col in cont_cols:
    train_df[col] = train_df[col].fillna(train_df[col].median())

In [ ]:
#  Label Encode Categoricals
# we add +1 later in the model to handle that -1 Missing code
for col in cat_cols:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col].astype(str))

In [ ]:
#scaling the cont columns here using standard scalar
scaler = StandardScaler()
train_df[cont_cols] = scaler.fit_transform(train_df[cont_cols])

In [ ]:
train_ds = ACSDataset(train_df, cat_cols, cont_cols, 'poverty_risk')
#  DataLoader handles shuffling/ batching
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)


In [ ]:

cards = [df[col].nunique() for col in cat_cols]

model = FTTransformer(
    cat_cardinalities=cards,
    n_cont=len(cont_cols),
    embed_dim=32
).to(device)

criterion = torch.nn.CrossEntropyLoss()
#is this the best optimizer? keep reading
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
model.train()
for epoch in range(10):
    #o
    epoch_loss = 0
    for x_cat, x_cont, y in train_loader:
        x_cat, x_cont, y = x_cat.to(device), x_cont.to(device), y.to(device)

        optimizer.zero_grad()
        output = model(x_cat, x_cont)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch {epoch+1} | Avg Loss: {epoch_loss/len(train_loader):.4f}")